In [ ]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [ ]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [ ]:
# set the path
file_path = "file:///home/talentum/test-jupyter/evaluation/Lab1/hortonworks.txt"
# read the path
fileRDD = sc.textFile(file_path)
# split on ','
splitRDD = fileRDD.map(lambda line: line.split(','))
# create flatmap on line[0] which is url as value and keys are remaining words
mapRDD = splitRDD.flatMap(lambda line: ((word, line[0]) for word in line[1:]))
# reduce by key
finalRDD = mapRDD.reduceByKey(lambda url1, url2: url1 + ',' + url2)
# print the key, value
for key, value in finalRDD.collect():
    print(key, value)

hadoop http://hortonworks.com/,http://hortonworks.com/download/,http://hortonworks.com/community/,http://hortonworks.com/kb,http://hortonworks.com/about-us/,http://hortonworks.com/resources/,http://hortonworks.com/webinars/,http://hortonworks.com/resources/,http://hortonworks.com/hadoop-training/
webinars http://hortonworks.com/
enterprise http://hortonworks.com/,http://hortonworks.com/products/hortonworksdataplatform/,http://hortonworks.com/about-us/contact-us/
team http://hortonworks.com/
reliability http://hortonworks.com/,http://hortonworks.com/resources/
feed http://hortonworks.com/products/,http://hortonworks.com/kb,http://hortonworks.com/resources/
board http://hortonworks.com/products/
password http://hortonworks.com/products/hortonworksdataplatform/,http://hortonworks.com/community/
hdp http://hortonworks.com/get-started/,http://hortonworks.com/download/,http://hortonworks.com/about-us/contact-us/,http://hortonworks.com/events/,http://hortonworks.com/webinars/,http://hortonwor

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("word", StringType(), True),
    StructField("urls", StringType(), True)
])

# Convert the RDD to a DataFrame
df = spark.createDataFrame(finalRDD, schema)

df.show()

+--------------+--------------------+
|          word|                urls|
+--------------+--------------------+
|        hadoop|http://hortonwork...|
|      webinars|http://hortonwork...|
|    enterprise|http://hortonwork...|
|          team|http://hortonwork...|
|   reliability|http://hortonwork...|
|          feed|http://hortonwork...|
|         board|http://hortonwork...|
|      password|http://hortonwork...|
|           hdp|http://hortonwork...|
| presentations|http://hortonwork...|
|       connect|http://hortonwork...|
| knowledgebase|http://hortonwork...|
|      platform|http://hortonwork...|
|       support|http://hortonwork...|
|      training|http://hortonwork...|
|       webinar|http://hortonwork...|
|instructor-led|http://hortonwork...|
|      articles|http://hortonwork...|
|      download|http://hortonwork...|
|   hortonworks|http://hortonwork...|
+--------------+--------------------+
only showing top 20 rows

